In [1]:
# This notebook reads in the discretised input data and then preprocesses the model features
# Firstly, values deemed excessively high/low are capped
# Relevant binary features and normally/log-normally features are standardised accordingly
# Training and test sets are split - 70% train, 10% validation, 20% test
# Resulting datasets are saved to file.


#import the file wehre he said

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame

In [3]:
# disc_inp_data = pd.read_csv("../data/discretised_input_data.csv")
disc_inp_data = pd.read_csv(r"../data/sepsis_final_data_withTimes.csv")
# dem

In [4]:
disc_inp_data.head()
print(disc_inp_data.count())


traj                   256098
step                   256098
m:presumed_onset       256098
m:charttime            256098
m:icustayid            256098
o:gender               256098
o:mechvent             256098
o:max_dose_vaso        256098
o:re_admission         256098
o:age                  256098
o:Weight_kg            256098
o:GCS                  256098
o:HR                   256098
o:SysBP                256098
o:MeanBP               256098
o:DiaBP                256098
o:RR                   256098
o:Temp_C               256098
o:FiO2_1               256098
o:Potassium            256098
o:Sodium               256098
o:Chloride             256098
o:Glucose              256098
o:Magnesium            256098
o:Calcium              256098
o:Hb                   256098
o:WBC_count            256098
o:Platelets_count      256098
o:PTT                  256098
o:PT                   256098
o:Arterial_pH          256098
o:paO2                 256098
o:paCO2                256098
o:Arterial

In [5]:
renaming_dict={}
for column_name in disc_inp_data.columns.values:
    renaming_dict[column_name]=column_name.replace("m:","").replace("o:","").replace("a:","").replace("r:","")

disc_inp_data.rename(columns = renaming_dict, inplace = True)

In [6]:
disc_inp_data.head()

,traj,step,presumed_onset,charttime,icustayid,gender,mechvent,max_dose_vaso,re_admission,age,...,SGOT,SGPT,Total_bili,INR,input_total,input_4hourly,output_total,output_4hourly,action,reward
0,1,0,7.245590e+09,7.245486e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,0.338959,0.157965,-0.500365,-0.218993,0.768941,0.261902,0.783054,0.687331,10,0.0
1,1,1,7.245590e+09,7.245500e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,-0.187354,-0.179660,1.685584,-0.959725,0.774064,0.261902,0.790869,0.579335,10,0.0
2,1,2,7.245590e+09,7.245515e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,-0.187354,-0.179660,1.685584,-0.959725,0.779148,0.261902,0.796727,0.498449,10,0.0
3,1,3,7.245590e+09,7.245529e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,-0.187354,-0.179660,1.685584,-0.218993,0.784194,0.261902,0.803687,0.557687,10,0.0
4,1,4,7.245590e+09,7.245544e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,-0.187354,-0.179660,1.685584,-0.218993,0.789203,0.261902,0.814416,0.698847,10,0.0


In [7]:
print(disc_inp_data.count())

traj                 256098
step                 256098
presumed_onset       256098
charttime            256098
icustayid            256098
gender               256098
mechvent             256098
max_dose_vaso        256098
re_admission         256098
age                  256098
Weight_kg            256098
GCS                  256098
HR                   256098
SysBP                256098
MeanBP               256098
DiaBP                256098
RR                   256098
Temp_C               256098
FiO2_1               256098
Potassium            256098
Sodium               256098
Chloride             256098
Glucose              256098
Magnesium            256098
Calcium              256098
Hb                   256098
WBC_count            256098
Platelets_count      256098
PTT                  256098
PT                   256098
Arterial_pH          256098
paO2                 256098
paCO2                256098
Arterial_BE          256098
HCO3                 256098
Arterial_lactate    

In [8]:
demog_data = pd.read_csv(r"../data/demog.csv",sep='|')

In [9]:
demog_data = demog_data.rename(columns= {'icustay_id':'icustayid'})
# demog_data = demog_data[['icustayid','morta_hosp', 'morta_90', 'elixhauser']]

demog_data.head()

,subject_id,hadm_id,icustayid,admittime,dischtime,adm_order,unit,intime,outtime,los,age,dob,dod,expire_flag,gender,morta_hosp,morta_90,elixhauser
0,2,163353,243653,5.318680e+09,5.319014e+09,1,5,5.318688e+09,5.318696e+09,0.0918,0.888970,5.318611e+09,NaN,0,1,NaN,NaN,0.0
1,3,145834,211552,4.159278e+09,4.160210e+09,1,1,4.159279e+09,4.159803e+09,6.0646,27950.798738,1.744330e+09,4.179686e+09,1,1,0.0,0.0,2.0
2,4,185777,294638,6.980517e+09,6.981187e+09,1,1,6.980517e+09,6.980662e+09,1.6785,17475.020498,5.470675e+09,NaN,0,2,NaN,NaN,3.0
3,5,178980,214757,4.199834e+09,4.200034e+09,1,5,4.199839e+09,4.199847e+09,0.0844,0.253056,4.199818e+09,NaN,0,1,NaN,NaN,0.0
4,6,107064,228232,6.482100e+09,6.483514e+09,1,2,6.482151e+09,6.482468e+09,3.6729,24084.896458,4.401216e+09,NaN,0,2,NaN,NaN,1.0


In [10]:
demog_data.count()

subject_id     61532
hadm_id        61532
icustayid      61532
admittime      61532
dischtime      61532
adm_order      61532
unit           61532
intime         61532
outtime        61522
los            61522
age            61532
dob            61532
dod            24191
expire_flag    61532
gender         61532
morta_hosp     24191
morta_90       24191
elixhauser     61215
dtype: int64

In [11]:
#Number of unique patients/subjects for those corresponding ICU stay id's

uniq_sub = pd.merge(disc_inp_data,demog_data,on='icustayid')
# print(uniq_sub['subject_id'].nunique())
print("Nbr of unique patients: ", uniq_sub['subject_id'].nunique())
print("Nbr of hospital admissions: ", uniq_sub['hadm_id'].nunique())
print("Nbr of ICU stays: ", disc_inp_data['icustayid'].nunique())
# print("Nbr of ICU stays: ", uniq_sub['icustayid'].nunique())

Nbr of unique patients:  16226
Nbr of hospital admissions:  19619
Nbr of ICU stays:  19619


In [12]:
# This notebook takes the input dataset, finds quartiles for the medical interventions (IV in, max vaso in)
# for each block.
# We then discretise actions in the original dataset according to what bin they fall in, and then save 
# a new dataframe with the discretised actions.


# Pasted from https://github.com/aniruddhraghu/sepsisrl/blob/master/preprocessing/process_interventions.ipynb

In [13]:
# new_data = pd.merge(disc_inp_data,demog_data, on='icustayid', how='left')
# new_data.count()

In [14]:
#Get all the other columns using demographic data left join

new_data1 = pd.merge(disc_inp_data,demog_data, on='icustayid', how='left')
new_data1.head()

,traj,step,presumed_onset,charttime,icustayid,gender_x,mechvent,max_dose_vaso,re_admission,age_x,...,outtime,los,age_y,dob,dod,expire_flag,gender_y,morta_hosp,morta_90,elixhauser
0,1,0,7.245590e+09,7.245486e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,7.245566e+09,5.8884,17639.826435,5.720976e+09,7.251552e+09,1,1,0.0,1.0,0.0
1,1,1,7.245590e+09,7.245500e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,7.245566e+09,5.8884,17639.826435,5.720976e+09,7.251552e+09,1,1,0.0,1.0,0.0
2,1,2,7.245590e+09,7.245515e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,7.245566e+09,5.8884,17639.826435,5.720976e+09,7.251552e+09,1,1,0.0,1.0,0.0
3,1,3,7.245590e+09,7.245529e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,7.245566e+09,5.8884,17639.826435,5.720976e+09,7.251552e+09,1,1,0.0,1.0,0.0
4,1,4,7.245590e+09,7.245544e+09,200003.0,-0.5,-0.5,-2.302585,-0.5,-0.975798,...,7.245566e+09,5.8884,17639.826435,5.720976e+09,7.251552e+09,1,1,0.0,1.0,0.0


In [15]:
new_data1.count()

traj              256098
step              256098
presumed_onset    256098
charttime         256098
icustayid         256098
                   ...  
expire_flag       256098
gender_y          256098
morta_hosp        124917
morta_90          124917
elixhauser        255965
Length: 71, dtype: int64

In [16]:
# df2 = df.sort_values(['Fee', 'Duration'],
#               ascending = [False, True])

new_data = new_data1.sort_values(['icustayid','charttime'],
              ascending = [True, True])

In [ ]:
# Run it only once!

run_it=True

# Add an END state to each trajectory. This takes some time, so comment out if already done!
new_data['END'] = False
new_df = pd.DataFrame(columns=new_data.columns)
new_idx = 0
if run_it:
    for i in new_data.index:
        if i == 0:
            new_df.loc[new_idx, new_data.columns] = new_data.loc[i,:]
            new_idx +=1
            continue
        if new_data.loc[i, 'icustayid'] != new_data.loc[i-1, 'icustayid']:
            # new patient; add an end state to the new_df
            new_df.loc[new_idx, new_data.columns] = np.nan
            new_df.loc[new_idx, 'icustayid'] = new_df.loc[new_idx-1, 'icustayid']
            new_df.loc[new_idx, 'END'] = True
            new_idx += 1
            new_df.loc[new_idx, new_data.columns] = new_data.loc[i,:]
            new_idx += 1
        else:
            new_df.loc[new_idx, new_data.columns] = new_data.loc[i,:]
            new_idx += 1
        if i % 1000 == 0:
            print(i)
    run_it=False
    new_data = new_df

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000


In [ ]:
# assume we're using input_4hourly and max_dose_vaso as the input params for now
interventions = new_data[["max_dose_vaso", "input_4hourly"]]

#changed 4hourly_tev to 4hourly

In [ ]:
interventions.head()

In [ ]:
# adjusted_vaso = interventions["max_dose_vaso"][interventions["max_dose_vaso"] >0]
# adjusted_iv = interventions["input_4hourly_tev"][interventions["input_4hourly_tev"]>0]

adjusted_vaso = interventions["max_dose_vaso"][interventions["max_dose_vaso"] >0]
adjusted_iv = interventions["input_4hourly"][interventions["input_4hourly"]>0]

In [ ]:
vaso_quartiles = adjusted_vaso.quantile([0.25,0.50,0.75])
iv_quartiles = adjusted_iv.quantile([0.25,0.5,0.75])

In [ ]:
vq = np.array(vaso_quartiles)
vq

In [ ]:
ivq = np.array(iv_quartiles)
ivq

In [ ]:
import copy
discretised_int = copy.deepcopy(interventions)

In [ ]:
discretised_int['vaso_input'] = discretised_int['max_dose_vaso']
discretised_int['vaso_input'][interventions['max_dose_vaso'] == 0.0] = 0
discretised_int['vaso_input'][(interventions['max_dose_vaso'] > 0.0) & (interventions['max_dose_vaso'] < vq[0])] = 1
discretised_int['vaso_input'][(interventions['max_dose_vaso'] >= vq[0]) & (interventions['max_dose_vaso'] < vq[1])] = 2
discretised_int['vaso_input'][(interventions['max_dose_vaso'] >= vq[1]) & (interventions['max_dose_vaso'] < vq[2])] = 3
a = interventions['max_dose_vaso'] >= vq[2]
discretised_int['vaso_input'][a] = 4

In [ ]:
discretised_int['iv_input'] = discretised_int['input_4hourly']
discretised_int['iv_input'][interventions['input_4hourly'] == 0.0] = 0
discretised_int['iv_input'][(interventions['input_4hourly'] > 0.0) & (interventions['input_4hourly'] < ivq[0])] = 1
discretised_int['iv_input'][(interventions['input_4hourly'] >=  ivq[0]) & (interventions['input_4hourly'] <  ivq[1])] = 2
discretised_int['iv_input'][(interventions['input_4hourly'] >=  ivq[1]) & (interventions['input_4hourly'] < ivq[2])] = 3
discretised_int['iv_input'][(interventions['input_4hourly'] >=  ivq[2])] = 4

In [ ]:
discretised_int['vaso_input'].plot.hist()

In [ ]:
discretised_int['iv_input'].plot.hist()

In [ ]:
disc_inp_data = copy.deepcopy(new_data)

In [ ]:
disc_inp_data['vaso_input'] = discretised_int['vaso_input']
disc_inp_data['iv_input'] = discretised_int['iv_input']

In [ ]:
disc_inp_data.head()

In [ ]:
disc_inp_data['vaso_input'].value_counts()

In [ ]:
disc_inp_data['iv_input'].value_counts()

In [ ]:
# disc_inp_data.to_csv('../data/discretised_input_data.csv', index=False)
# disc_inp_data = pd.read_csv("../data/discretised_input_data.csv")
disc_inp_data.to_csv(r"../data/discretised_input_data.csv', index=False)

In [ ]:
disc_inp_data['reward'].value_counts()

In [ ]:
my_list = list(disc_inp_data)

print (my_list)

In [ ]:
disc_inp_data['morta_hosp'].value_counts()

### Going back to the preprocess_data.ipynb

In [ ]:
disc_inp_data = pd.read_csv(r"../data/discretised_input_data.csv")

In [ ]:
# Since we don't have died_in_hosp, using morta_hosp & renaming the column

disc_inp_data.rename(columns={'morta_hosp': 'died_in_hosp'}, inplace=True)

In [ ]:
my_list = list(disc_inp_data)

print (my_list)

In [ ]:
# add rewards - sparsely for now; reward function shaping comes in a separate script
disc_inp_data['reward'] = 0
for i in disc_inp_data.index:
    if i == 0:
        continue
    else:
        if disc_inp_data.loc[i, 'icustayid'] != disc_inp_data.loc[i-1, 'icustayid']:
            if disc_inp_data.loc[i-1, 'died_in_hosp'] == 1:
                disc_inp_data.loc[i-1,'reward'] = -100
            elif disc_inp_data.loc[i-1, 'died_in_hosp'] == 0:
                disc_inp_data.loc[i-1,'reward'] = 100
            else:
                print("error in row", i-1)
if disc_inp_data.loc[len(disc_inp_data)-1, 'died_in_hosp'] == 1:
    disc_inp_data.loc[len(disc_inp_data)-1, 'reward'] = -100
elif disc_inp_data.loc[len(disc_inp_data)-1, 'died_in_hosp'] == 0:
     disc_inp_data.loc[len(disc_inp_data)-1, 'reward'] = 100
print(disc_inp_data['reward'].value_counts())

In [ ]:
# now split into train/validation/test sets
import random
unique_ids = disc_inp_data['icustayid'].unique()
random.shuffle(unique_ids)
train_sample = 0.7
val_sample = 0.1
test_sample = 0.2
train_num = int(len(unique_ids) * 0.7)
val_num = int(len(unique_ids)*0.1) + train_num
train_ids = unique_ids[:train_num]
val_ids = unique_ids[train_num:val_num]
test_ids = unique_ids[val_num:]

In [ ]:
train_set = DataFrame()
train_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(train_ids)]

val_set = DataFrame()
val_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(val_ids)]

test_set = DataFrame()
test_set = disc_inp_data.loc[disc_inp_data['icustayid'].isin(test_ids)]

In [ ]:
# cap values in train and test
caps = pd.read_csv("capping_values.csv")
for i in caps.index:
    param = caps.loc[i,'parameter'][1:-1]
    maxval = caps.loc[i,'limsup']
    minval = caps.loc[i,'liminf']
    train_set[param][train_set[param] >= maxval] = maxval
    train_set[param][train_set[param] <= minval] = minval
    val_set[param][val_set[param] >= maxval] = maxval
    val_set[param][val_set[param] <= minval] = minval
    test_set[param][test_set[param] >= maxval] = maxval
    test_set[param][test_set[param] <= minval] = minval

In [ ]:
binary_fields = ['gender','mechvent','re_admission']
norm_fields= ['age','Weight_kg','GCS','HR','SysBP','MeanBP','DiaBP','RR','Temp_C','FiO2_1',
    'Potassium','Sodium','Chloride','Glucose','Magnesium','Calcium',
    'Hb','WBC_count','Platelets_count','PTT','PT','Arterial_pH','paO2','paCO2',
    'Arterial_BE','HCO3','Arterial_lactate','SOFA','SIRS','Shock_Index',
    'PaO2_FiO2','cumulated_balance_tev', 'elixhauser', 'Albumin', u'CO2_mEqL', 'Ionised_Ca']
log_fields = ['max_dose_vaso','SpO2','BUN','Creatinine','SGOT','SGPT','Total_bili','INR',
              'input_total_tev','input_4hourly_tev','output_total','output_4hourly', 'bloc']

In [ ]:
# normalise binary fields
train_set[binary_fields] = train_set[binary_fields] - 0.5 
val_set[binary_fields] = val_set[binary_fields] - 0.5 
test_set[binary_fields] = test_set[binary_fields] - 0.5 

In [ ]:
# normal distn fields
for item in norm_fields:
    av = train_set[item].mean()
    std = train_set[item].std()
    train_set[item] = (train_set[item] - av) / std
    val_set[item] = (val_set[item] - av) / std
    test_set[item] = (test_set[item] - av) / std

In [ ]:
# log normal fields
train_set[log_fields] = np.log(0.1 + train_set[log_fields])
val_set[log_fields] = np.log(0.1 + val_set[log_fields])
test_set[log_fields] = np.log(0.1 + test_set[log_fields])
for item in log_fields:
    av = train_set[item].mean()
    std = train_set[item].std()
    train_set[item] = (train_set[item] - av) / std
    val_set[item] = (val_set[item] - av) / std
    test_set[item] = (test_set[item] - av) / std

In [ ]:
train_set.head()

In [ ]:
train_set.to_csv('../data/rl_train_set_unscaled.csv',index = False)
val_set.to_csv('../data/rl_val_set_unscaled.csv', index = False)
test_set.to_csv('../data/rl_test_set_unscaled.csv', index = False)

In [ ]:
# scale features to [0,1] in train set, similar in val and test
import copy
scalable_fields = copy.deepcopy(binary_fields)
scalable_fields.extend(norm_fields)
scalable_fields.extend(log_fields)
for col in scalable_fields:
    minimum = min(train_set[col])
    maximum = max(train_set[col])
    train_set[col] = (train_set[col] - minimum)/(maximum-minimum)
    val_set[col] = (val_set[col] - minimum)/(maximum-minimum)
    test_set[col] = (test_set[col] - minimum)/(maximum-minimum)

In [ ]:
train_set.head()

In [ ]:
train_set.to_csv('../data/rl_train_set_scaled.csv',index = False)
val_set.to_csv('../data/rl_val_set_scaled.csv', index = False)
test_set.to_csv('../data/rl_test_set_scaled.csv', index = False)